<a href="https://colab.research.google.com/github/Snizhana-DA/A-B-Testing-Analysis-Statistical-Significance/blob/main/Portfolio_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Виведення перших 5 рядків датасету
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import mannwhitneyu
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/Pandas
df = pd.read_csv("ab_test.csv")
df.head()

Mounted at /content/drive
/content/drive/MyDrive/Pandas


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new_account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new_account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new_account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new_account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new_account,1


## Розрахунок статистичної значущості чотирьох метрик в тоталі для тесту 1.

In [ ]:
# Виведення результатів тесту 1 в тоталі
filtered_df = df[df["test"] == 1]
grouped_df1 = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()  # Скидаємо індекс, щоб event_name став колонкою
# Перейменовуємо стовпці
grouped_df1.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1)

             event_name  test_group1  test_group2
0      add_payment_info         1988         2229
1     add_shipping_info         3034         3221
2           add_to_cart         1395         1366
3        begin_checkout         3784         4021
4                 click          368          353
5           first_visit        30596        30512
6           new_account         3823         3681
7             page_view       191543       198050
8                scroll        73244        73376
9           select_item          543          530
10     select_promotion         1275         1323
11              session        45362        45193
12        session_start        45905        45649
13  session_with_orders         4514         4526
14      user_engagement       171788       179081
15            view_item        62335        65337
16       view_item_list           27           24
17       view_promotion        29188        29117
18  view_search_results         3678         3882


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df1["test_group1"].sum()
total_group2 = grouped_df1["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 49.37%
Пропорція групи 2: 50.63%
Розподіл в групах є рівномірним.


In [ ]:
# Розрахунок статистичної значущості метрики: add_payment_info / session
# Витягуємо кількість сесій та add_payment_info для обох груп
session_1 = grouped_df1.loc[grouped_df1["event_name"]=="session", "test_group1"].iloc[0]  # number of session for group 1
session_2 = grouped_df1.loc[grouped_df1["event_name"]=="session", "test_group2"].iloc[0]  # number of session for group 2
add_payment_info_1 = grouped_df1.loc[grouped_df1["event_name"]=="add_payment_info", "test_group1"].iloc[0]   # number of add_payment_info for group 1
add_payment_info_2 = grouped_df1.loc[grouped_df1["event_name"]=="add_payment_info", "test_group2"].iloc[0]  # number of add_payment_info for group 2
# Обчислюємо конверсії
conversion_rate_1 = add_payment_info_1 / session_1
conversion_rate_2 = add_payment_info_2 / session_2
print(f"Conversion Rate Group 1: {conversion_rate_1:.2%}")
print(f"Conversion Rate Group 2: {conversion_rate_2:.2%}")
metric_change = conversion_rate_2 / conversion_rate_1 - 1
print(f"Metric_change: {metric_change:.2%}")
# Виконуємо Z-тест
z_stat, p_value = sm.stats.proportions_ztest([add_payment_info_1, add_payment_info_2], [session_1, session_2])
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")
# Інтерпретація результатів
if p_value < 0.05:
  significant = True
else:
    significant = False
print(f"Significant: {significant}")

Conversion Rate Group 1: 4.38%
Conversion Rate Group 2: 4.93%
Metric_change: 12.54%
Z-statistic: -3.924884023129521
P-value: 8.677153267271231e-05
Significant: True


**Висновок:**
Оскільки p-value дуже малий, ми відкидаємо нульову гіпотезу (H₀), що між групами немає різниці. Z-statistic - від'ємне значення, означає, що група 2 має вищу конверсію Add Payment Info щодо кількості сесій, ніж група 1. Зміни, застосовані до групи 2, мали позитивний ефект.


In [ ]:
# Розрахунок статистичної значущості метрики: add_shipping_info / session

# Витягуємо кількість сесій та add_shipping_info для обох груп
session_1 = grouped_df1.loc[grouped_df1["event_name"]=="session", "test_group1"].iloc[0]  # number of session for group 1
session_2 = grouped_df1.loc[grouped_df1["event_name"]=="session", "test_group2"].iloc[0]  # number of session for group 2
add_shipping_info_1 = grouped_df1.loc[grouped_df1["event_name"]=="add_shipping_info", "test_group1"].iloc[0]   # number of add_shipping_info for group 1
add_shipping_info_2 = grouped_df1.loc[grouped_df1["event_name"]=="add_shipping_info", "test_group2"].iloc[0]  # number of add_shipping_info for group 2
# Обчислюємо конверсії
conversion_rate_1 = add_shipping_info_1 / session_1
conversion_rate_2 = add_shipping_info_2 / session_2
print(f"Conversion Rate Group 1: {conversion_rate_1:.2%}")
print(f"Conversion Rate Group 2: {conversion_rate_2:.2%}")
metric_change = conversion_rate_2 / conversion_rate_1 - 1
print(f"Metric_change: {metric_change:.2%}")
# Виконуємо Z-тест
z_stat, p_value = sm.stats.proportions_ztest([add_shipping_info_1, add_shipping_info_2], [session_1, session_2])
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")
# Інтерпретація результатів
if p_value < 0.05:
  significant = True
else:
    significant = False
print(f"Significant: {significant}")

Conversion Rate Group 1: 6.69%
Conversion Rate Group 2: 7.13%
Metric_change: 6.56%
Z-statistic: -2.603571454721395
P-value: 0.009225803404204286
Significant: True


**Висновок:**
Група 2 має вищу конверсію Add shipping Info щодо кількості сесій, ніж група 1. Зміни, застосовані до групи 2, мали позитивний ефект.

In [ ]:
# Розрахунок статистичної значущості метрики: begin_checkout / session

# Витягуємо кількість сесій та begin_checkout для обох груп
session_1 = grouped_df1.loc[grouped_df1["event_name"]=="session", "test_group1"].iloc[0]  # number of session for group 1
session_2 = grouped_df1.loc[grouped_df1["event_name"]=="session", "test_group2"].iloc[0]  # number of session for group 2
begin_checkout_1 = grouped_df1.loc[grouped_df1["event_name"]=="begin_checkout", "test_group1"].iloc[0]   # number of begin_checkout for group 1
begin_checkout_2 = grouped_df1.loc[grouped_df1["event_name"]=="begin_checkout", "test_group2"].iloc[0]  # number of begin_checkout for group 2
# Обчислюємо конверсії
conversion_rate_1 = begin_checkout_1 / session_1
conversion_rate_2 = begin_checkout_2 / session_2
print(f"Conversion Rate Group 1: {conversion_rate_1:.2%}")
print(f"Conversion Rate Group 2: {conversion_rate_2:.2%}")
metric_change = conversion_rate_2 / conversion_rate_1 - 1
print(f"Metric_change: {metric_change:.2%}")
# Виконуємо Z-тест
z_stat, p_value = sm.stats.proportions_ztest([begin_checkout_1, begin_checkout_2], [session_1, session_2])
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")
# Інтерпретація результатів
if p_value < 0.05:
  significant = True
else:
    significant = False
print(f"Significant: {significant}")

Conversion Rate Group 1: 8.34%
Conversion Rate Group 2: 8.90%
Metric_change: 6.66%
Z-statistic: -2.978782961833959
P-value: 0.0028939568500496436
Significant: True


**Висновок:**
Група 2 має вищу конверсію Begin Checkout щодо кількості сесій, ніж група 1. Зміни, застосовані до групи 2, мали позитивний ефект.

In [ ]:
# Розрахунок статистичної значущості метрики: new_accounts / session
# Витягуємо кількість сесій та new_accounts для обох груп
session_1 = grouped_df1.loc[grouped_df1["event_name"]=="session", "test_group1"].iloc[0]  # number of session for group 1
session_2 = grouped_df1.loc[grouped_df1["event_name"]=="session", "test_group2"].iloc[0]  # number of session for group 2
new_account_1 = grouped_df1.loc[grouped_df1["event_name"]=="new_account", "test_group1"].iloc[0]   # number of new_accounts for group 1
new_account_2 = grouped_df1.loc[grouped_df1["event_name"]=="new_account", "test_group2"].iloc[0]  # number of new_accounts for group 2
# Обчислюємо конверсії
conversion_rate_1 = new_account_1 / session_1
conversion_rate_2 = new_account_2 / session_2
print(f"Conversion Rate Group 1: {conversion_rate_1:.2%}")
print(f"Conversion Rate Group 2: {conversion_rate_2:.2%}")
metric_change = conversion_rate_2 / conversion_rate_1 - 1
print(f"Metric_change: {metric_change:.2%}")
# Виконуємо Z-тест
z_stat, p_value = sm.stats.proportions_ztest([new_account_1, new_account_2], [session_1, session_2])
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")
# Інтерпретація результатів
if p_value < 0.05:
  significant = True
else:
    significant = False
print(f"Significant: {significant}")

Conversion Rate Group 1: 8.43%
Conversion Rate Group 2: 8.15%
Metric_change: -3.35%
Z-statistic: 1.542882614576384
P-value: 0.1228592597741769
Significant: False


**Висновок:**
Метрика new account / session просіла в допустимих діапазонах, результат не є статистично значущим.

### Спрощення коду за допомогою циклу.

In [ ]:
# Список метрик, які потрібно обробити
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Створення списків для обчислення сесій та кількості подій для кожної групи
for metric in metrics:
    # Витягуємо кількість сесій та метрики для обох груп
    session_1 = grouped_df1.loc[grouped_df1["event_name"] == "session", "test_group1"].iloc[0]  # number of session for group 1
    session_2 = grouped_df1.loc[grouped_df1["event_name"] == "session", "test_group2"].iloc[0]  # number of session for group 2
    event_1 = grouped_df1.loc[grouped_df1["event_name"] == metric, "test_group1"].iloc[0]  # number of event for group 1
    event_2 = grouped_df1.loc[grouped_df1["event_name"] == metric, "test_group2"].iloc[0]  # number of event for group 2
    # Обчислюємо конверсії для кожної метрики
    conversion_rate_1 = event_1 / session_1
    conversion_rate_2 = event_2 / session_2
    print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
    print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
    metric_change = conversion_rate_2 / conversion_rate_1 - 1
    print(f"Metric_change: {metric_change:.2%}")
    # Виконуємо Z-тест для кожної метрики
    z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
    print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    # Інтерпретація результатів
    significant = p_value < 0.05
    print(f"Significant for {metric}: {'True' if significant else 'False'}")
    print("-" * 50)  # Роздільник для кращої читабельності

Conversion Rate Add_payment_info Group 1: 4.38%
Conversion Rate Add_payment_info Group 2: 4.93%
Metric_change: 12.54%
Z-statistic for Add_payment_info: -3.924884023129521
P-value for Add_payment_info: 8.677153267271231e-05
Significant for add_payment_info: True
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 6.69%
Conversion Rate Add_shipping_info Group 2: 7.13%
Metric_change: 6.56%
Z-statistic for Add_shipping_info: -2.603571454721395
P-value for Add_shipping_info: 0.009225803404204286
Significant for add_shipping_info: True
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 8.34%
Conversion Rate Begin_checkout Group 2: 8.90%
Metric_change: 6.66%
Z-statistic for Begin_checkout: -2.978782961833959
P-value for Begin_checkout: 0.0028939568500496436
Significant for begin_checkout: True
--------------------------------------------------
Conversion Rate New_account Group 1: 8.43%
Conversion Rate New_accoun

**Висновок:** Результати не змінилися, використання циклу значно спрощує код і додає гнучкість.

## Розрахунок статистичної значущості чотирьох метрик в тоталі для тесту 2.

In [ ]:
# Виведення результатів тесту 2 в тоталі
filtered_df = df[df["test"] == 2]
grouped_df2 = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()  # Скидаємо індекс, щоб event_name став колонкою
# Перейменовуємо стовпці
grouped_df2.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df2)

             event_name  test_group1  test_group2
0      add_payment_info         2344         2409
1     add_shipping_info         3480         3510
2           add_to_cart         2811         3061
3        begin_checkout         4262         4313
4                 click          337          413
5           first_visit        34511        34171
6           new_account         4165         4184
7             page_view       220275       212320
8                scroll        80713        81370
9           select_item          905          946
10     select_promotion         1477         1406
11              session        50637        50244
12        session_start        51219        50808
13  session_with_orders         5102         5003
14      user_engagement       198266       189931
15            view_item        72717        68700
16       view_item_list           24           29
17       view_promotion        32367        31680
18  view_search_results         4282         4198


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df2["test_group1"].sum()
total_group2 = grouped_df2["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 50.70%
Пропорція групи 2: 49.30%
Розподіл в групах є рівномірним.


In [ ]:
# Список метрик, які потрібно обробити
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Створення списків для обчислення сесій та кількості подій для кожної групи
for metric in metrics:
    # Витягуємо кількість сесій та метрики для обох груп
    session_1 = grouped_df2.loc[grouped_df1["event_name"] == "session", "test_group1"].iloc[0]  # number of session for group 1
    session_2 = grouped_df2.loc[grouped_df1["event_name"] == "session", "test_group2"].iloc[0]  # number of session for group 2
    event_1 = grouped_df2.loc[grouped_df1["event_name"] == metric, "test_group1"].iloc[0]  # number of event for group 1
    event_2 = grouped_df2.loc[grouped_df1["event_name"] == metric, "test_group2"].iloc[0]  # number of event for group 2
    # Обчислюємо конверсії для кожної метрики
    conversion_rate_1 = event_1 / session_1
    conversion_rate_2 = event_2 / session_2
    print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
    print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
    metric_change = conversion_rate_2 / conversion_rate_1 - 1
    print(f"Metric_change: {metric_change:.2%}")
    # Виконуємо Z-тест для кожної метрики
    z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
    print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    # Інтерпретація результатів
    significant = p_value < 0.05
    print(f"Significant for {metric}: {'True' if significant else 'False'}")
    print("-" * 50)  # Роздільник для кращої читабельності

Conversion Rate Add_payment_info Group 1: 4.63%
Conversion Rate Add_payment_info Group 2: 4.79%
Metric_change: 3.58%
Z-statistic for Add_payment_info: -1.240993925082954
P-value for Add_payment_info: 0.2146079930564252
Significant for add_payment_info: False
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 6.87%
Conversion Rate Add_shipping_info Group 2: 6.99%
Metric_change: 1.65%
Z-statistic for Add_shipping_info: -0.7095568649433848
P-value for Add_shipping_info: 0.4779789765684105
Significant for add_shipping_info: False
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 8.42%
Conversion Rate Begin_checkout Group 2: 8.58%
Metric_change: 1.99%
Z-statistic for Begin_checkout: -0.9528980407686692
P-value for Begin_checkout: 0.3406417328714627
Significant for begin_checkout: False
--------------------------------------------------
Conversion Rate New_account Group 1: 8.23%
Conversion Rate New_account Gr

**Висновок:**
Результати A/B тесту 2 показують, що жодна з протестованих метрик не має статистично значущої різниці між Group 1 і Group 2, хоча і спостерігається позитивний вплив.
- Conversion Rate Add_payment_info: Конверсія зросла на 3.58%, але p-value = 0.2146, що означає відсутність статистичної значущості.
- Conversion Rate Add_shipping_info: Збільшення конверсії на 1.65%, але p-value = 0.4780 вказує на те, що ефект несуттєвий.
- Conversion Rate Begin_checkout: Конверсія зросла на 1.99%, однак p-value = 0.3406, що не свідчить про статистичну значущість.
- Conversion Rate New_account: Незначне збільшення на 1.24%, але p-value = 0.5560, що підтверджує відсутність значущого впливу.

Оскільки всі значення p-value перевищують поріг 0.05, ми не можемо відхилити нульову гіпотезу. Це означає, що зафіксовані відмінності між Group 1 і Group 2, можуть бути випадковими і не мати реального впливу. Для отримання більш точних висновків можна провести додаткове тестування, оскільки спостерігається, хоч і незначна, позитивна тенденція по всіх метриках.




## Розрахунок статистичної значущості чотирьох метрик в тоталі для тесту 3.

In [ ]:
# Виведення результатів тесту 3 в тоталі
filtered_df = df[df["test"] == 3]
grouped_df3 = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()  # Скидаємо індекс, щоб event_name став колонкою
# Перейменовуємо стовпці
grouped_df3.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df3)

             event_name  test_group1  test_group2
0      add_payment_info         3623         3697
1     add_shipping_info         5298         5188
2           add_to_cart        17674        17229
3        begin_checkout         9532         9264
4                 click          280          292
5           first_visit        50438        50728
6           new_account         5856         5822
7             page_view       286351       284287
8                scroll       110360       109564
9           select_item         8735         8556
10     select_promotion         2020         1996
11              session        70047        70439
12        session_start        71312        71686
13  session_with_orders         6951         7151
14      user_engagement       249921       248325
15            view_item        93931        92841
16       view_item_list            9            9
17       view_promotion        41169        40959
18  view_search_results         5764         5619


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df3["test_group1"].sum()
total_group2 = grouped_df3["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 50.14%
Пропорція групи 2: 49.86%
Розподіл в групах є рівномірним.


In [ ]:
# Список метрик, які потрібно обробити
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Створення списків для обчислення сесій та кількості подій для кожної групи
for metric in metrics:
    # Витягуємо кількість сесій та метрики для обох груп
    session_1 = grouped_df3.loc[grouped_df1["event_name"] == "session", "test_group1"].iloc[0]  # number of session for group 1
    session_2 = grouped_df3.loc[grouped_df1["event_name"] == "session", "test_group2"].iloc[0]  # number of session for group 2
    event_1 = grouped_df3.loc[grouped_df1["event_name"] == metric, "test_group1"].iloc[0]  # number of event for group 1
    event_2 = grouped_df3.loc[grouped_df1["event_name"] == metric, "test_group2"].iloc[0]  # number of event for group 2
    # Обчислюємо конверсії для кожної метрики
    conversion_rate_1 = event_1 / session_1
    conversion_rate_2 = event_2 / session_2
    print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
    print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
    metric_change = conversion_rate_2 / conversion_rate_1 - 1
    print(f"Metric_change: {metric_change:.2%}")
    # Виконуємо Z-тест для кожної метрики
    z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
    print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    # Інтерпретація результатів
    significant = p_value < 0.05
    print(f"Significant for {metric}: {'True' if significant else 'False'}")
    print("-" * 50)  # Роздільник для кращої читабельності

Conversion Rate Add_payment_info Group 1: 5.17%
Conversion Rate Add_payment_info Group 2: 5.25%
Metric_change: 1.47%
Z-statistic for Add_payment_info: -0.6431724006448517
P-value for Add_payment_info: 0.5201122396904715
Significant for add_payment_info: False
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 7.56%
Conversion Rate Add_shipping_info Group 2: 7.37%
Metric_change: -2.62%
Z-statistic for Add_shipping_info: 1.4137271445347053
P-value for Add_shipping_info: 0.15744203211836594
Significant for add_shipping_info: False
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 13.61%
Conversion Rate Begin_checkout Group 2: 13.15%
Metric_change: -3.35%
Z-statistic for Begin_checkout: 2.5113885009036334
P-value for Begin_checkout: 0.012025726872364946
Significant for begin_checkout: True
--------------------------------------------------
Conversion Rate New_account Group 1: 8.36%
Conversion Rate New_accou

**Висновок:**
Результати A/B тесту 3 показують, що лише одна з протестованих метрик має негативний результат та статистично значущу різницю між Group 1 і Group 2.
- Conversion Rate Add_payment_info: Конверсія зросла на 1.47%, але p-value = 0.5201, що означає відсутність статистичної значущості.
- Conversion Rate Add_shipping_info: Конверсія знизилася на 2.62%, але p-value = 0.1574, що не досягає рівня статистичної значущості.
- Conversion Rate Begin_checkout: Конверсія знизилася на 3.35%, при цьому p-value = 0.0120, що є статистично значущим (p < 0.05). Це вказує на те, що зміни застосовані до групи 2 мали негативний вплив.
- Conversion Rate New_account: Незначне зниження на 1.13%, але p-value = 0.5199, що свідчить про відсутність статистичної значущості.

Оскільки для Conversion Rate Begin_checkout значення p-value < 0.05, ми відхиляємо нульову гіпотезу, що означає наявність значущого просідання для цієї метрики. Для всіх інших метрик статистично значущих змін не зафіксовано, і спостережувані відмінності можуть бути випадковими.


## Розрахунок статистичної значущості чотирьох метрик в тоталі для тесту 4.

In [ ]:
# Виведення результатів тесту 4 в тоталі
filtered_df = df[df["test"] == 4]
grouped_df4 = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()  # Скидаємо індекс, щоб event_name став колонкою
# Перейменовуємо стовпці
grouped_df4.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df4)

             event_name  test_group1  test_group2
0      add_payment_info         3731         3601
1     add_shipping_info         5128         4956
2           add_to_cart        21536        21554
3        begin_checkout        12555        12267
4                 click          285          299
5           first_visit        80900        80626
6           new_account         8984         8687
7             page_view       379480       378307
8                scroll       136037       134156
9           select_item        12214        11988
10     select_promotion         2773         2708
11              session       105079       105141
12        session_start       107128       106978
13  session_with_orders        10596        10481
14      user_engagement       273946       273633
15            view_item        98619        98928
16       view_item_list            5            6
17       view_promotion        52672        51985
18  view_search_results         7582         7203


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df4["test_group1"].sum()
total_group2 = grouped_df4["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 50.11%
Пропорція групи 2: 49.89%
Розподіл в групах є рівномірним.


In [ ]:
# Список метрик, які потрібно обробити
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Створення списків для обчислення сесій та кількості подій для кожної групи
for metric in metrics:
    # Витягуємо кількість сесій та метрики для обох груп
    session_1 = grouped_df4.loc[grouped_df1["event_name"] == "session", "test_group1"].iloc[0]  # number of session for group 1
    session_2 = grouped_df4.loc[grouped_df1["event_name"] == "session", "test_group2"].iloc[0]  # number of session for group 2
    event_1 = grouped_df4.loc[grouped_df1["event_name"] == metric, "test_group1"].iloc[0]  # number of event for group 1
    event_2 = grouped_df4.loc[grouped_df1["event_name"] == metric, "test_group2"].iloc[0]  # number of event for group 2
    # Обчислюємо конверсії для кожної метрики
    conversion_rate_1 = event_1 / session_1
    conversion_rate_2 = event_2 / session_2
    print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
    print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
    metric_change = conversion_rate_2 / conversion_rate_1 - 1
    print(f"Metric_change: {metric_change:.2%}")
    # Виконуємо Z-тест для кожної метрики
    z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
    print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    # Інтерпретація результатів
    significant = p_value < 0.05
    print(f"Significant for {metric}: {'True' if significant else 'False'}")
    print("-" * 50)  # Роздільник для кращої читабельності

Conversion Rate Add_payment_info Group 1: 3.55%
Conversion Rate Add_payment_info Group 2: 3.42%
Metric_change: -3.54%
Z-statistic for Add_payment_info: 1.5711064424503227
P-value for Add_payment_info: 0.11615792640849863
Significant for add_payment_info: False
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 4.88%
Conversion Rate Add_shipping_info Group 2: 4.71%
Metric_change: -3.41%
Z-statistic for Add_shipping_info: 1.785795361681634
P-value for Add_shipping_info: 0.07413239802026593
Significant for add_shipping_info: False
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 11.95%
Conversion Rate Begin_checkout Group 2: 11.67%
Metric_change: -2.35%
Z-statistic for Begin_checkout: 1.9959978123649886
P-value for Begin_checkout: 0.04593416091816525
Significant for begin_checkout: True
--------------------------------------------------
Conversion Rate New_account Group 1: 8.55%
Conversion Rate New_accoun

**Висновок:**
Результати A/B тесту 4 показують, просідання всіх чотирьох мертик, дві з яких мають статистично значущі відмінності між Group 1 і Group 2.
- Conversion Rate Add_payment_info: Конверсія знизилася на 3.54%, але p-value = 0.1162, що означає відсутність статистичної значущості.
- Conversion Rate Add_shipping_info: Конверсія зменшилася на 3.41, p-value = 0.0741, результати не значущі.
- Conversion Rate Begin_checkout: Конверсія знизилася на 2.35%, при цьому p-value = 0.0459, що є статистично значущим (p < 0.05). Це свідчить про те, що різниця між групами не є випадковою.
- Conversion Rate New_account: Конверсія знизилася на 3.36%, при цьому p-value = 0.0175, що є статистично значущим (p < 0.05).

Оскільки p-value < 0.05 для Conversion Rate Begin_checkout і Conversion Rate New_account, ми відхиляємо нульову гіпотезу для цих метрик. Це означає, що відмінності між Group 1 і Group 2 для цих показників є статистично значущими.
Для Conversion Rate Add_payment_info і Conversion Rate Add_shipping_info значення p-value > 0.05, тому ми не можемо відхилити нульову гіпотезу, і спостережувані зміни, можуть бути випадковими. Оскільки зміни, застосовані до групи 2 мали негативний вплив по всіх метриках, для двох з яких результати  були статистично значущими, тест не можемо вважати успішним.



## Розрахунок статистичної значущості чотирьох метрик для тесту 1 в розрізі каналів.

In [ ]:
# Виведення результатів тесту 1 для каналу Direct
filtered_df = df[(df["test"]==1) & (df["channel"] == "Direct")]
grouped_df1_direct = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_direct.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_direct)

             event_name  test_group1  test_group2
0      add_payment_info          392          516
1     add_shipping_info          664          716
2           add_to_cart          269          279
3        begin_checkout          823          915
4                 click           79           73
5           first_visit         7432         7283
6           new_account          913          850
7             page_view        44951        45157
8                scroll        17041        17141
9           select_item          104          126
10     select_promotion          316          307
11              session        10691        10361
12        session_start        10875        10532
13  session_with_orders         1084         1052
14      user_engagement        40283        40604
15            view_item        14644        14476
16       view_item_list            6            4
17       view_promotion         6985         6667
18  view_search_results          868          876


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df1_direct["test_group1"].sum()
total_group2 = grouped_df1_direct["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 50.08%
Пропорція групи 2: 49.92%
Розподіл в групах є рівномірним.


In [ ]:
# Список метрик, які потрібно обробити
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Створення списків для обчислення сесій та кількості подій для кожної групи
for metric in metrics:
    # Витягуємо кількість сесій та метрики для обох груп
    session_1 = grouped_df1_direct.loc[grouped_df1_direct["event_name"] == "session", "test_group1"].iloc[0]  # number of session for group 1
    session_2 = grouped_df1_direct.loc[grouped_df1_direct["event_name"] == "session", "test_group2"].iloc[0]  # number of session for group 2
    event_1 = grouped_df1_direct.loc[grouped_df1_direct["event_name"] == metric, "test_group1"].iloc[0]  # number of event for group 1
    event_2 = grouped_df1_direct.loc[grouped_df1_direct["event_name"] == metric, "test_group2"].iloc[0]  # number of event for group 2
    # Обчислюємо конверсії для кожної метрики
    conversion_rate_1 = event_1 / session_1
    conversion_rate_2 = event_2 / session_2
    print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
    print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
    metric_change = conversion_rate_2 / conversion_rate_1 - 1
    print(f"Metric_change: {metric_change:.2%}")
    # Виконуємо Z-тест для кожної метрики
    z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
    print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    # Інтерпретація результатів
    significant = p_value < 0.05
    print(f"Significant for {metric.capitalize()}: {'True' if significant else 'False'}")
    print("-" * 50)  # Роздільник для кращої читабельності

Conversion Rate Add_payment_info Group 1: 3.67%
Conversion Rate Add_payment_info Group 2: 4.98%
Metric_change: 35.83%
Z-statistic for Add_payment_info: -4.690261152959848
P-value for Add_payment_info: 2.7285656989392044e-06
P-value for Add_payment_info: 2.7285656989392044e-06
Significant for Add_payment_info: True
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 6.21%
Conversion Rate Add_shipping_info Group 2: 6.91%
Metric_change: 11.27%
Z-statistic for Add_shipping_info: -2.050707497700552
P-value for Add_shipping_info: 0.040295440378197364
P-value for Add_shipping_info: 0.040295440378197364
Significant for Add_shipping_info: True
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 7.70%
Conversion Rate Begin_checkout Group 2: 8.83%
Metric_change: 14.72%
Z-statistic for Begin_checkout: -2.986589005359636
P-value for Begin_checkout: 0.0028210870432094437
P-value for Begin_checkout: 0.0028210870432094437

**Висновок:**
Результати A/B тесту 1 для каналу Direct показують, що три з чотирьох метрик мають позитивні статистично значущі відмінності між Group 1 і Group 2.
- Conversion Rate Add_payment_info: Конверсія зросла на 35.83%, при цьому p-value = 2.73e-06, що є статистично значущим (p < 0.05). Це вказує на значний позитивний вплив змін у Group 2.
- Conversion Rate Add_shipping_info: Конверсія зросла на 11.27%, при цьому p-value = 0.0403, що є статистично значущим (p < 0.05). Це свідчить про те, що зміни в Group 2 мали позитивний вплив на цей показник.
- Conversion Rate Begin_checkout: Конверсія збільшилася на 14.72%, при цьому p-value = 0.0028, що є статистично значущим (p < 0.05). Це вказує на суттєву різницю між групами.
- Conversion Rate New_account: Конверсія зменшилася на 3.94%, але p-value = 0.3789, що означає відсутність статистичної значущості (p > 0.05).

Оскільки p-value < 0.05 для Conversion Rate Add_payment_info, Conversion Rate Add_shipping_info та Conversion Rate Begin_checkout, ми відхиляємо нульову гіпотезу для цих метрик. Це означає, що Group 2 показала статистично значуще покращення в порівнянні з Group 1 для цих показників.
Для Conversion Rate New_account значення p-value > 0.05, тому ми не можемо відхилити нульову гіпотезу, і спостережуване просідання цієї метрики, ймовірно, є випадковими.


In [ ]:
# Виведення результатів тесту 1 для каналу Organic Search
filtered_df = df[(df["test"]==1) & (df["channel"] == "Organic Search")]
grouped_df1_os = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_os.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_os)

             event_name  test_group1  test_group2
0      add_payment_info          640          514
1     add_shipping_info         1021          922
2           add_to_cart          494          392
3        begin_checkout         1249         1144
4                 click           90           87
5           first_visit        12438        12305
6           new_account         1307         1301
7             page_view        69237        69391
8                scroll        25561        23822
9           select_item          207          152
10     select_promotion          442          461
11              session        15675        15631
12        session_start        15963        15903
13  session_with_orders         1502         1534
14      user_engagement        61525        62108
15            view_item        22584        22786
16       view_item_list            7            6
17       view_promotion        10274        10287
18  view_search_results         1349         1413


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df1_os["test_group1"].sum()
total_group2 = grouped_df1_os["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 50.15%
Пропорція групи 2: 49.85%
Розподіл в групах є рівномірним.


In [ ]:
# Список метрик, які потрібно обробити
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Створення списків для обчислення сесій та кількості подій для кожної групи
for metric in metrics:
    # Витягуємо кількість сесій та метрики для обох груп
    session_1 = grouped_df1_os.loc[grouped_df1_os["event_name"] == "session", "test_group1"].iloc[0]  # number of session for group 1
    session_2 = grouped_df1_os.loc[grouped_df1_os["event_name"] == "session", "test_group2"].iloc[0]  # number of session for group 2
    event_1 = grouped_df1_os.loc[grouped_df1_os["event_name"] == metric, "test_group1"].iloc[0]  # number of event for group 1
    event_2 = grouped_df1_os.loc[grouped_df1_os["event_name"] == metric, "test_group2"].iloc[0]  # number of event for group 2
    # Обчислюємо конверсії для кожної метрики
    conversion_rate_1 = event_1 / session_1
    conversion_rate_2 = event_2 / session_2
    print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
    print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
    metric_change = conversion_rate_2 / conversion_rate_1 - 1
    print(f"Metric_change: {metric_change:.2%}")
    # Виконуємо Z-тест для кожної метрики
    z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
    print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    # Інтерпретація результатів
    significant = p_value < 0.05
    print(f"Significant for {metric.capitalize()}: {'True' if significant else 'False'}")
    print("-" * 50)  # Роздільник для кращої читабельності

Conversion Rate Add_payment_info Group 1: 4.08%
Conversion Rate Add_payment_info Group 2: 3.29%
Metric_change: -19.46%
Z-statistic for Add_payment_info: 3.730758025866682
P-value for Add_payment_info: 0.0001909045129292049
P-value for Add_payment_info: 0.0001909045129292049
Significant for Add_payment_info: True
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 6.51%
Conversion Rate Add_shipping_info Group 2: 5.90%
Metric_change: -9.44%
Z-statistic for Add_shipping_info: 2.2550944824879924
P-value for Add_shipping_info: 0.024127398901353108
P-value for Add_shipping_info: 0.024127398901353108
Significant for Add_shipping_info: True
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 7.97%
Conversion Rate Begin_checkout Group 2: 7.32%
Metric_change: -8.15%
Z-statistic for Begin_checkout: 2.1619556233036095
P-value for Begin_checkout: 0.030621596905618862
P-value for Begin_checkout: 0.030621596905618862
Sig

**Висновок:**
Результати A/B тесту 1 для каналу Organic Search показують, що три з чотирьох метрик мають статистично значущі негативні відмінності між групами.
- Conversion Rate Add_payment_info: Конверсія знизилася на 19.46%, при цьому p-value = 0.00019, що є статистично значущим (p < 0.05). Це свідчить про значне зниження показника в Group 2.
-	Conversion Rate Add_shipping_info: Конверсія зменшилася на 9.44%, при цьому p-value = 0.0241, що є статистично значущим (p < 0.05). Це означає, що зміни в Group 2 мали негативний вплив.
- Conversion Rate Begin_checkout: Конверсія знизилася на 8.15%, при цьому p-value = 0.0306, що є статистично значущим (p < 0.05). Це також вказує на суттєву різницю між групами.
- Conversion Rate New_account: Незначне зменшення на 0.18%, але p-value = 0.9619, що означає відсутність статистичної значущості (p > 0.05).

Оскільки p-value < 0.05 для Conversion Rate Add_payment_info, Conversion Rate Add_shipping_info та Conversion Rate Begin_checkout, ми відхиляємо нульову гіпотезу для цих метрик. Це означає, що Group 2 показала статистично значуще зниження цих показників у порівнянні з Group 1.
Для Conversion Rate New_account значення p-value > 0.05, тому ми не можемо відхилити нульову гіпотезу, і спостережувані зміни можуть бути випадковими.
Загалом результати свідчать про негативний вплив змін у Group 2, що може вимагати подальшого аналізу та перегляду змін.



In [ ]:
# Виведення результатів тесту 1 для каналу Social Search
filtered_df = df[(df["test"]==1) & (df["channel"] == "Social Search")]
grouped_df1_ss = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_ss.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_ss)

             event_name  test_group1  test_group2
0      add_payment_info          237          264
1     add_shipping_info          300          353
2           add_to_cart          120          198
3        begin_checkout          392          435
4                 click           51           48
5           first_visit         1492         1578
6           new_account          312          335
7             page_view        15580        17351
8                scroll         6638         6914
9           select_item           61           38
10     select_promotion          101          109
11              session         3883         3963
12        session_start         3861         3924
13  session_with_orders          375          416
14      user_engagement        14306        16140
15            view_item         5168         6023
16       view_item_list            2            9
17       view_promotion         2329         2437
18  view_search_results          316          315


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df1_ss["test_group1"].sum()
total_group2 = grouped_df1_ss["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 47.71%
Пропорція групи 2: 52.29%
Розподіл в групах не рівномірний.


Значення пропорцій близьке до очікуваних діапазонів, тому можемо вважати розподіл рівномірним.

In [ ]:
# Список метрик, які потрібно обробити
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Створення списків для обчислення сесій та кількості подій для кожної групи
for metric in metrics:
    # Витягуємо кількість сесій та метрики для обох груп
    session_1 = grouped_df1_ss.loc[grouped_df1_ss["event_name"] == "session", "test_group1"].iloc[0]  # number of session for group 1
    session_2 = grouped_df1_ss.loc[grouped_df1_ss["event_name"] == "session", "test_group2"].iloc[0]  # number of session for group 2
    event_1 = grouped_df1_ss.loc[grouped_df1_ss["event_name"] == metric, "test_group1"].iloc[0]  # number of event for group 1
    event_2 = grouped_df1_ss.loc[grouped_df1_ss["event_name"] == metric, "test_group2"].iloc[0]  # number of event for group 2

    # Обчислюємо конверсії для кожної метрики
    conversion_rate_1 = event_1 / session_1
    conversion_rate_2 = event_2 / session_2
    print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
    print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
    metric_change = conversion_rate_2 / conversion_rate_1 - 1
    print(f"Metric_change: {metric_change:.2%}")

    # Виконуємо Z-тест для кожної метрики
    z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
    print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    # Інтерпретація результатів
    significant = p_value < 0.05
    print(f"Significant for {metric.capitalize()}: {'True' if significant else 'False'}")
    print("-" * 50)  # Роздільник для кращої читабельності

Conversion Rate Add_payment_info Group 1: 6.10%
Conversion Rate Add_payment_info Group 2: 6.66%
Metric_change: 9.14%
Z-statistic for Add_payment_info: -1.0109059440882084
P-value for Add_payment_info: 0.31206144881364284
P-value for Add_payment_info: 0.31206144881364284
Significant for Add_payment_info: False
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 7.73%
Conversion Rate Add_shipping_info Group 2: 8.91%
Metric_change: 15.29%
Z-statistic for Add_shipping_info: -1.8941234885607305
P-value for Add_shipping_info: 0.058208620675484775
P-value for Add_shipping_info: 0.058208620675484775
Significant for Add_shipping_info: False
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 10.10%
Conversion Rate Begin_checkout Group 2: 10.98%
Metric_change: 8.73%
Z-statistic for Begin_checkout: -1.2709444175976554
P-value for Begin_checkout: 0.20374842306540475
P-value for Begin_checkout: 0.20374842306540475
Sign

**Висновок:**
Результати A/B тесту 1 для каналу Social Search показують, що всі чотири метрики не мають статистично значущих відмінностей між Group 1 і Group 2.
- Conversion Rate Add_payment_info: Конверсія зросла на 9.14%, але p-value = 0.3121, що не є статистично значущим (p > 0.05). Це означає, що спостережене зростання може бути випадковим.
- Conversion Rate Add_shipping_info: Конверсія збільшилася на 15.29%, при цьому p-value = 0.0582, що трохи вище порогового значення 0.05. Це означає, що різниця між групами не є достатньо значущою з точки зору статистики.
- Conversion Rate Begin_checkout: Конверсія зросла на 8.73%, але p-value = 0.2037, що не є статистично значущим (p > 0.05).
- Conversion Rate New_account: Конверсія зросла на 5.20%, але p-value = 0.5008, що також не є статистично значущим (p > 0.05).

Оскільки для всіх чотирьох метрик p-value > 0.05, ми не можемо відхилити нульову гіпотезу. Це означає, що спостережені зміни в конверсіях між Group 1 і Group 2 не є статистично значущими, і ймовірно, можуть бути результатом випадкових коливань. Загалом зміни в Group 2 не продемонстрували значущого  впливу на конверсію, проте для отримання більш точних висновків можна провести додаткове тестування, оскільки спостерігається, позитивна тенденція по всіх метриках.



In [ ]:
# Виведення результатів тесту 1 для каналу Paid Search
filtered_df = df[(df["test"]==1) & (df["channel"] == "Paid Search")]
grouped_df1_ps = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_ps.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_ps)

             event_name  test_group1  test_group2
0      add_payment_info          450          519
1     add_shipping_info          770          794
2           add_to_cart          379          365
3        begin_checkout          967          958
4                 click           91           78
5           first_visit         9195         9295
6           new_account          990          948
7             page_view        49040        50966
8                scroll        18127        18530
9           select_item          123          128
10     select_promotion          338          352
11              session        11777        11841
12        session_start        11981        11990
13  session_with_orders         1185         1183
14      user_engagement        43429        45767
15            view_item        15638        16503
16       view_item_list            8            5
17       view_promotion         7777         7692
18  view_search_results          890         1012


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df1_ps["test_group1"].sum()
total_group2 = grouped_df1_ps["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 49.18%
Пропорція групи 2: 50.82%
Розподіл в групах є рівномірним.


In [ ]:
# Список метрик, які потрібно обробити
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Створення списків для обчислення сесій та кількості подій для кожної групи
for metric in metrics:
    # Витягуємо кількість сесій та метрики для обох груп
    session_1 = grouped_df1_ps.loc[grouped_df1_ps["event_name"] == "session", "test_group1"].iloc[0]  # number of session for group 1
    session_2 = grouped_df1_ps.loc[grouped_df1_ps["event_name"] == "session", "test_group2"].iloc[0]  # number of session for group 2
    event_1 = grouped_df1_ps.loc[grouped_df1_ps["event_name"] == metric, "test_group1"].iloc[0]  # number of event for group 1
    event_2 = grouped_df1_ps.loc[grouped_df1_ps["event_name"] == metric, "test_group2"].iloc[0]  # number of event for group 2
    # Обчислюємо конверсії для кожної метрики
    conversion_rate_1 = event_1 / session_1
    conversion_rate_2 = event_2 / session_2
    print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
    print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
    metric_change = conversion_rate_2 / conversion_rate_1 - 1
    print(f"Metric_change: {metric_change:.2%}")
    # Виконуємо Z-тест для кожної метрики
    z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
    print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    print(f"P-value for {metric.capitalize()}: {p_value}")
    # Інтерпретація результатів
    significant = p_value < 0.05
    print(f"Significant for {metric.capitalize()}: {'True' if significant else 'False'}")
    print("-" * 50)  # Роздільник для кращої читабельності

Conversion Rate Add_payment_info Group 1: 3.82%
Conversion Rate Add_payment_info Group 2: 4.38%
Metric_change: 14.71%
Z-statistic for Add_payment_info: -2.177389322371911
P-value for Add_payment_info: 0.029451531682692326
P-value for Add_payment_info: 0.029451531682692326
Significant for Add_payment_info: True
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 6.54%
Conversion Rate Add_shipping_info Group 2: 6.71%
Metric_change: 2.56%
Z-statistic for Add_shipping_info: -0.5171175484031051
P-value for Add_shipping_info: 0.6050741043001047
P-value for Add_shipping_info: 0.6050741043001047
Significant for Add_shipping_info: False
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 8.21%
Conversion Rate Begin_checkout Group 2: 8.09%
Metric_change: -1.47%
Z-statistic for Begin_checkout: 0.3380931746905805
P-value for Begin_checkout: 0.7352929738262673
P-value for Begin_checkout: 0.7352929738262673
Significant 

**Висновок:**
Результати тесту 1 для каналу Paid Search показують, що лише одна з чотирьох метрик має статистично значущі відмінності між Group 1 і Group 2.
- Conversion Rate Add_payment_info: Конверсія зросла на 14.71%, p-value = 0.0295, що є статистично значущим (p < 0.05). Це означає, що зміни в Group 2 мали позитивний вплив на цей показник.
- Conversion Rate Add_shipping_info: Незначне зростання на 2.56%, але p-value = 0.6051, що не є статистично значущим (p > 0.05). Це свідчить про те, що зміни могли бути випадковими.
- Conversion Rate Begin_checkout: Незначне зниження на 1.47%, p-value = 0.7353, що також не є статистично значущим (p > 0.05).
- Conversion Rate New_account: Зниження на 4.76%, але p-value = 0.2626, що не є статистично значущим (p > 0.05).

Оскільки p-value < 0.05 тільки для Conversion Rate Add_payment_info, ми можемо відхилити нульову гіпотезу і визнати, що зміни в Group 2 суттєво покращили цей показник. Для інших метрик p-value > 0.05, тому ми не можемо стверджувати, що зміни в Group 2 вплинули на них. Загалом, тест показав позитивний ефект для Add_payment_info, але для решти метрик статистично значущих відмінностей не виявлено.



### Спрощення коду для роботи з різним каналами та метриками.

In [ ]:
# Список метрик для обробки
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Словник для каналу з відповідними дата-сетами
datasets = {
    'direct': grouped_df1_direct,
    'organic_search': grouped_df1_os,
    'social_search': grouped_df1_ss,
    'paid_search': grouped_df1_ps
}
# Перебір по всіх каналах і метриках
for channel, data in datasets.items():
    print(f"\nResults for channel: {channel.capitalize()}")
    for metric in metrics:
        # Витягуємо кількість сесій та метрики для обох груп
        session_1 = data.loc[data["event_name"] == "session", "test_group1"].iloc[0]
        session_2 = data.loc[data["event_name"] == "session", "test_group2"].iloc[0]
        event_1 = data.loc[data["event_name"] == metric, "test_group1"].iloc[0]
        event_2 = data.loc[data["event_name"] == metric, "test_group2"].iloc[0]
        # Обчислюємо конверсії для кожної метрики
        conversion_rate_1 = event_1 / session_1
        conversion_rate_2 = event_2 / session_2
        print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
        print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
        metric_change = conversion_rate_2 / conversion_rate_1 - 1
        print(f"Metric_change {metric.capitalize()}: {metric_change:.2%}")
        # Виконуємо Z-тест для кожної метрики
        z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
        print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
        print(f"P-value for {metric.capitalize()}: {p_value}")
        print(f"P-value for {metric.capitalize()}: {p_value}")
        # Інтерпретація результатів
        significant = p_value < 0.05
        print(f"Significant for {metric.capitalize()}: {'True' if significant else 'False'}")
        print("-" * 50)  # Роздільник для кращої читабельності


Results for channel: Direct
Conversion Rate Add_payment_info Group 1: 3.67%
Conversion Rate Add_payment_info Group 2: 4.98%
Metric_change Add_payment_info: 35.83%
Z-statistic for Add_payment_info: -4.690261152959848
P-value for Add_payment_info: 2.7285656989392044e-06
P-value for Add_payment_info: 2.7285656989392044e-06
Significant for Add_payment_info: True
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 6.21%
Conversion Rate Add_shipping_info Group 2: 6.91%
Metric_change Add_shipping_info: 11.27%
Z-statistic for Add_shipping_info: -2.050707497700552
P-value for Add_shipping_info: 0.040295440378197364
P-value for Add_shipping_info: 0.040295440378197364
Significant for Add_shipping_info: True
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 7.70%
Conversion Rate Begin_checkout Group 2: 8.83%
Metric_change Begin_checkout: 14.72%
Z-statistic for Begin_checkout: -2.986589005359636
P-value for Begin_ch

## Розрахунок статистичної значущості чотирьох метрик для тесту 1 в розрізі пристроїв.

In [ ]:
# Виведення результатів тесту 1 для пристроїв Desktop
filtered_df = df[(df["test"]==1) & (df["device"] == "desktop")]
grouped_df1_desktop = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_desktop.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_desktop)

             event_name  test_group1  test_group2
0      add_payment_info         1130         1256
1     add_shipping_info         1711         1916
2           add_to_cart          821          824
3        begin_checkout         2108         2404
4                 click          246          237
5           first_visit        17999        18068
6           new_account         2203         2147
7             page_view       109466       116972
8                scroll        42185        43170
9           select_item          315          331
10     select_promotion          774          794
11              session        26467        26417
12        session_start        26966        26902
13  session_with_orders         2678         2677
14      user_engagement        98144       105638
15            view_item        35281        38386
16       view_item_list           16           13
17       view_promotion        16976        17142
18  view_search_results         2116         2326


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df1_desktop["test_group1"].sum()
total_group2 = grouped_df1_desktop["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 48.74%
Пропорція групи 2: 51.26%
Розподіл в групах є рівномірним.


In [ ]:
# Виведення результатів тесту 1 для пристроїв Mobile
filtered_df = df[(df["test"]==1) & (df["device"] == "mobile")]
grouped_df1_mobile = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_mobile.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_mobile)

             event_name  test_group1  test_group2
0      add_payment_info          810          942
1     add_shipping_info         1257         1256
2           add_to_cart          537          511
3        begin_checkout         1593         1561
4                 click          109          110
5           first_visit        11898        11749
6           new_account         1530         1441
7             page_view        77248        77264
8                scroll        29308        28766
9           select_item          202          187
10     select_promotion          472          499
11              session        17896        17767
12        session_start        17907        17726
13  session_with_orders         1738         1771
14      user_engagement        69634        70005
15            view_item        25362        25826
16       view_item_list           11           11
17       view_promotion        11552        11324
18  view_search_results         1476         1479


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df1_mobile["test_group1"].sum()
total_group2 = grouped_df1_mobile["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 50.03%
Пропорція групи 2: 49.97%
Розподіл в групах є рівномірним.


In [ ]:
# Виведення результатів тесту 1 для пристроїв Tablet
filtered_df = df[(df["test"]==1) & (df["device"] == "tablet")]
grouped_df1_tablet = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_tablet.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_tablet)

             event_name  test_group1  test_group2
0      add_payment_info           48           31
1     add_shipping_info           66           49
2           add_to_cart           37           31
3        begin_checkout           83           56
4                 click           13            6
5           first_visit          699          695
6           new_account           90           93
7             page_view         4829         3814
8                scroll         1751         1440
9           select_item           26           12
10     select_promotion           29           30
11              session          999         1009
12        session_start         1032         1021
13  session_with_orders           98           78
14      user_engagement         4010         3438
15            view_item         1692         1125
16       view_promotion          660          651
17  view_search_results           86           77


In [ ]:
# Перевірка розподілу даних в групах
total_group1 = grouped_df1_tablet["test_group1"].sum()
total_group2 = grouped_df1_tablet["test_group2"].sum()
total = total_group1 + total_group2
proportion_group1 = total_group1 / total
proportion_group2 = total_group2 / total
print(f"Пропорція групи 1: {proportion_group1:.2%}")
print(f"Пропорція групи 2: {proportion_group2:.2%}")
if 0.48 <= proportion_group1 <= 0.52:
    print("Розподіл в групах є рівномірним.")
else:
    print("Розподіл в групах не рівномірний.")

Пропорція групи 1: 54.33%
Пропорція групи 2: 45.67%
Розподіл в групах не рівномірний.


In [ ]:
# Список метрик для обробки
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Словник з відповідними дата-сетами
datasets = {
    'desktop': grouped_df1_desktop,
    'mobile': grouped_df1_mobile,
    'tablet': grouped_df1_tablet
}
# Перебір по всіх пристроях і метриках
for device, data in datasets.items():
    print(f"\nResults for device: {device.capitalize()}")
    for metric in metrics:
        # Витягуємо кількість сесій та метрики для обох груп
        session_1 = data.loc[data["event_name"] == "session", "test_group1"].iloc[0]
        session_2 = data.loc[data["event_name"] == "session", "test_group2"].iloc[0]
        event_1 = data.loc[data["event_name"] == metric, "test_group1"].iloc[0]
        event_2 = data.loc[data["event_name"] == metric, "test_group2"].iloc[0]
        # Обчислюємо конверсії для кожної метрики
        conversion_rate_1 = event_1 / session_1
        conversion_rate_2 = event_2 / session_2
        print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
        print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
        metric_change = conversion_rate_2 / conversion_rate_1 - 1
        print(f"Metric_change {metric.capitalize()}: {metric_change:.2%}")
        # Виконуємо Z-тест для кожної метрики
        z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
        print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
        print(f"P-value for {metric.capitalize()}: {p_value}")
        print(f"P-value for {metric.capitalize()}: {p_value}")
        # Інтерпретація результатів
        significant = p_value < 0.05
        print(f"Significant for {metric.capitalize()}: {'True' if significant else 'False'}")
        print("-" * 50)  # Роздільник для кращої читабельності


Results for device: Desktop
Conversion Rate Add_payment_info Group 1: 4.27%
Conversion Rate Add_payment_info Group 2: 4.75%
Metric_change Add_payment_info: 11.36%
Z-statistic for Add_payment_info: -2.6869978643818566
P-value for Add_payment_info: 0.007209741371215498
P-value for Add_payment_info: 0.007209741371215498
Significant for Add_payment_info: True
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 6.46%
Conversion Rate Add_shipping_info Group 2: 7.25%
Metric_change Add_shipping_info: 12.19%
Z-statistic for Add_shipping_info: -3.5860236942640444
P-value for Add_shipping_info: 0.0003357584652017076
P-value for Add_shipping_info: 0.0003357584652017076
Significant for Add_shipping_info: True
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 7.96%
Conversion Rate Begin_checkout Group 2: 9.10%
Metric_change Begin_checkout: 14.26%
Z-statistic for Begin_checkout: -4.673979974671707
P-value for Begin_ch

**Висновок за результатами A/B тесту 1 в розрізі пристроїв:**

*Desktop*

На десктопах тест показав статистично значущі покращення для трьох з чотирьох метрик:
- Conversion Rate Add_payment_info: зростання на 11.36%, p-value = 0.0072 (значущо).
- Conversion Rate Add_shipping_info: зростання на 12.19%, p-value = 0.0003 (значущо).
- Conversion Rate Begin_checkout: зростання на 14.26%, p-value < 0.0001 (значущо).
- Conversion Rate New_account: зміни на -2.36%, p-value = 0.4115, не значущо.

*Mobile*

На мобільних пристроях значущі результати отримано лише для Add_payment_info, інші метрики не показали статистично значущих змін:
- Conversion Rate Add_payment_info: зростання на 17.14%, p-value = 0.0007 (значущо).
- Conversion Rate Add_shipping_info: незначне зростання на 0.65%, p-value = 0.8671, не значущо.
- Conversion Rate Begin_checkout: незначне зниження на -1.30%, p-value = 0.7010, не значущо.
- Conversion Rate New_account: зниження на -5.13%, p-value = 0.1337, не значущо.

*Tablet*

Для планшетів характерним був нерівномірний розподіл користувачів в групах, до того ж даних дуже мало, що могло вплинути на результат. Результати були негативними, з статистично значущим погіршенням для Add_payment_info та Begin_checkout:
- Conversion Rate Add_payment_info: зниження на -36.06%, p-value = 0.0459 (значущо).
- Conversion Rate Add_shipping_info: зниження на -26.49%, p-value = 0.0915, не значущо.
- Conversion Rate Begin_checkout: зниження на -33.20%, p-value = 0.0149 (значущо).
- Conversion Rate New_account: зміни на 2.31%, p-value = 0.8713, не значущо.

Загальні висновки:
- Найбільше позитивних змін відбулося на десктопах: зростання Add_payment_info, Add_shipping_info та Begin_checkout.
- На мобільних пристроях покращився лише Add_payment_info, інші метрики не змінилися значущо.
- На планшетах тест показав негативний вплив на Add_payment_info та Begin_checkout, а інші метрики не зазнали значущих змін.

Ці результати можуть свідчити про те, що зміни працюють краще для десктопної версії, але можуть негативно впливати на користувачів планшетів. Рекомендується детальніше дослідити взаємодію користувачів з сайтом на планшетах.


## Розрахунок статистичної значущості чотирьох метрик для тесту 1 в розрізі континентів.

In [ ]:
# Виведення результатів тесту 1 для континенту Europe
filtered_df = df[(df["test"]==1) & (df["continent"] == "Europe")]
grouped_df1_eu = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_eu.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_eu)

             event_name  test_group1  test_group2
0      add_payment_info          324          452
1     add_shipping_info          528          606
2           add_to_cart          214          303
3        begin_checkout          645          744
4                 click           71           82
5           first_visit         5789         5793
6           new_account          700          683
7             page_view        35639        36298
8                scroll        13354        13399
9           select_item           91          109
10     select_promotion          261          247
11              session         8472         8423
12        session_start         8572         8535
13  session_with_orders          828          860
14      user_engagement        31962        32639
15            view_item        11214        11436
16       view_item_list            4            3
17       view_promotion         5605         5488
18  view_search_results          597          667


In [ ]:
# Виведення результатів тесту 1 для континенту Americas
filtered_df = df[(df["test"]==1) & (df["continent"] == "Americas")]
grouped_df1_am = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_am.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_am)

             event_name  test_group1  test_group2
0      add_payment_info         1107         1177
1     add_shipping_info         1704         1738
2           add_to_cart          838          756
3        begin_checkout         2130         2164
4                 click          213          165
5           first_visit        16879        16732
6           new_account         2165         1971
7             page_view       107050       110318
8                scroll        41062        40325
9           select_item          290          275
10     select_promotion          674          725
11              session        25164        24769
12        session_start        25452        25024
13  session_with_orders         2505         2454
14      user_engagement        96138        99944
15            view_item        35351        37052
16       view_item_list           19           20
17       view_promotion        16060        16059
18  view_search_results         2131         2318


In [ ]:
# Виведення результатів тесту 1 для континенту Asia
filtered_df = df[(df["test"]==1) & (df["continent"] == "Asia")]
grouped_df1_as = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_as.columns = ["event_name", "test_group1", "test_group2"]
grouped_df1_as["test_group1"] = grouped_df1_as["test_group1"].fillna(0).astype(int)
grouped_df1_as["test_group2"] = grouped_df1_as["test_group2"].fillna(0).astype(int)
print(grouped_df1_as)

             event_name  test_group1  test_group2
0      add_payment_info          512          545
1     add_shipping_info          730          798
2           add_to_cart          299          274
3        begin_checkout          915         1002
4                 click           77           93
5           first_visit         7220         7274
6           new_account          878          934
7             page_view        44342        47023
8                scroll        17155        17665
9           select_item          156          122
10     select_promotion          302          327
11              session        10626        10932
12        session_start        10782        11006
13  session_with_orders         1072         1101
14      user_engagement        39641        42544
15            view_item        14357        15424
16       view_item_list            4            0
17       view_promotion         6797         6902
18  view_search_results          886          846


In [ ]:
# Виведення результатів тесту 1 для континенту Africa
filtered_df = df[(df["test"]==1) & (df["continent"] == "Africa")]
grouped_df1_af = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_af.columns = ["event_name", "test_group1", "test_group2"]
print(grouped_df1_af)

             event_name  test_group1  test_group2
0      add_payment_info           19           24
1     add_shipping_info           41           28
2           add_to_cart           16           14
3        begin_checkout           54           33
4                 click            2            6
5           first_visit          325          317
6           new_account           36           38
7             page_view         2118         1783
8                scroll          805          857
9           select_item            6            6
10     select_promotion           20            9
11              session          494          474
12        session_start          496          486
13  session_with_orders           46           45
14      user_engagement         1885         1575
15            view_item          618          564
16       view_promotion          364          282
17  view_search_results           25           18


In [ ]:
# Виведення результатів тесту 1 для континенту Oceania
filtered_df = df[(df["test"]==1) & (df["continent"] == "Oceania")]
grouped_df1_oc = filtered_df.groupby(["event_name", "test_group"])["value"].sum().unstack().reset_index()
grouped_df1_oc.columns = ["event_name", "test_group1", "test_group2"]
grouped_df1_oc["test_group1"] = grouped_df1_oc["test_group1"].fillna(0).astype(int)
grouped_df1_oc["test_group2"] = grouped_df1_oc["test_group2"].fillna(0).astype(int)
print(grouped_df1_oc)

             event_name  test_group1  test_group2
0      add_payment_info           19           21
1     add_shipping_info           27           38
2           add_to_cart           25           18
3        begin_checkout           36           56
4                 click            4            6
5           first_visit          318          330
6           new_account           37           44
7             page_view         2131         2028
8                scroll          743          953
9           select_item            0           14
10     select_promotion           17           14
11              session          509          495
12        session_start          505          495
13  session_with_orders           52           56
14      user_engagement         1926         1818
15            view_item          724          672
16       view_item_list            0            1
17       view_promotion          312          318
18  view_search_results           34           29


In [ ]:
# Перевірка рівномірності розподілу даних  в групах для кожного континенту.
def check_distribution(data, continent):
    total_group1 = data["test_group1"].sum()
    total_group2 = data["test_group2"].sum()
    total = total_group1 + total_group2
    proportion_group1 = total_group1 / total
    proportion_group2 = total_group2 / total
    print(f"\nРезультати для {continent}:")
    print(f"Пропорція групи 1: {proportion_group1:.2%}")
    print(f"Пропорція групи 2: {proportion_group2:.2%}")
    if 0.48 <= proportion_group1 <= 0.52:
        print("Розподіл в групах є рівномірним.")
    else:
        print("Розподіл в групах не рівномірний.")
# Список континентів і відповідних датасетів
datasets = {
    "Europe": grouped_df1_eu,
    "Americas": grouped_df1_am,
    "Asia": grouped_df1_as,
    "Africa": grouped_df1_af,
    "Oceania": grouped_df1_oc
}
# Перевіряємо рівномірність розподілу для кожного континенту.
for continent, df in datasets.items():
    check_distribution(df, continent)


Результати для Europe:
Пропорція групи 1: 49.62%
Пропорція групи 2: 50.38%
Розподіл в групах є рівномірним.

Результати для Americas:
Пропорція групи 1: 49.54%
Пропорція групи 2: 50.46%
Розподіл в групах є рівномірним.

Результати для Asia:
Пропорція групи 1: 48.75%
Пропорція групи 2: 51.25%
Розподіл в групах є рівномірним.

Результати для Africa:
Пропорція групи 1: 52.91%
Пропорція групи 2: 47.09%
Розподіл в групах не рівномірний.

Результати для Oceania:
Пропорція групи 1: 50.04%
Пропорція групи 2: 49.96%
Розподіл в групах є рівномірним.


In [ ]:
# Список метрик для обробки
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']
# Словник з відповідними дата-сетами
datasets = {
    "Europe": grouped_df1_eu,
    "Americas": grouped_df1_am,
    "Asia": grouped_df1_as,
    "Africa": grouped_df1_af,
    "Oceania": grouped_df1_oc
}
# Перебір по всіх континентах і метриках
for continent, data in datasets.items():
    print(f"\nResults for continent: {continent.capitalize()}")
    for metric in metrics:
        # Витягуємо кількість сесій та метрики для обох груп
        session_1 = data.loc[data["event_name"] == "session", "test_group1"].iloc[0]
        session_2 = data.loc[data["event_name"] == "session", "test_group2"].iloc[0]
        event_1 = data.loc[data["event_name"] == metric, "test_group1"].iloc[0]
        event_2 = data.loc[data["event_name"] == metric, "test_group2"].iloc[0]
        # Обчислюємо конверсії для кожної метрики
        conversion_rate_1 = event_1 / session_1
        conversion_rate_2 = event_2 / session_2
        print(f"Conversion Rate {metric.capitalize()} Group 1: {conversion_rate_1:.2%}")
        print(f"Conversion Rate {metric.capitalize()} Group 2: {conversion_rate_2:.2%}")
        metric_change = conversion_rate_2 / conversion_rate_1 - 1
        print(f"Metric_change {metric.capitalize()}: {metric_change:.2%}")
        # Виконуємо Z-тест для кожної метрики
        z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
        print(f"Z-statistic for {metric.capitalize()}: {z_stat}")
        print(f"P-value for {metric.capitalize()}: {p_value}")
        print(f"P-value for {metric.capitalize()}: {p_value}")
        # Інтерпретація результатів
        significant = p_value < 0.05
        print(f"Significant for {metric.capitalize()}: {'True' if significant else 'False'}")
        print("-" * 50)  # Роздільник для кращої читабельності


Results for continent: Europe
Conversion Rate Add_payment_info Group 1: 3.82%
Conversion Rate Add_payment_info Group 2: 5.37%
Metric_change Add_payment_info: 40.32%
Z-statistic for Add_payment_info: -4.7869707211827635
P-value for Add_payment_info: 1.6931752371532812e-06
P-value for Add_payment_info: 1.6931752371532812e-06
Significant for Add_payment_info: True
--------------------------------------------------
Conversion Rate Add_shipping_info Group 1: 6.23%
Conversion Rate Add_shipping_info Group 2: 7.19%
Metric_change Add_shipping_info: 15.44%
Z-statistic for Add_shipping_info: -2.4992734199700153
P-value for Add_shipping_info: 0.012444825223304968
P-value for Add_shipping_info: 0.012444825223304968
Significant for Add_shipping_info: True
--------------------------------------------------
Conversion Rate Begin_checkout Group 1: 7.61%
Conversion Rate Begin_checkout Group 2: 8.83%
Metric_change Begin_checkout: 16.02%
Z-statistic for Begin_checkout: -2.8856068631773115
P-value for Beg

**Висновок за результатами A/B тесту 1 в розрізі континентів:**

*Європа*

Тест показав статистично значущі покращення для трьох метрик:
- Add_payment_info: зростання на 40.32%, p-value < 0.0001 (значущо).
- Add_shipping_info: зростання на 15.44%, p-value = 0.0124 (значущо).
- Begin_checkout: зростання на 16.02%, p-value = 0.0039 (значущо).
- New_account: зміни на -1.86%, p-value = 0.7155, не значущо.

*Америка*

Для жодної метрики не виявлено значущих покращень, проте спостерігається негативний вплив на реєстрацію нових акаунтів:
- Add_payment_info: зростання на 8.02%, p-value = 0.0592, не значущо.
- Add_shipping_info: зростання на 3.62%, p-value = 0.2794, не значущо.
- Begin_checkout: зростання на 3.22%, p-value = 0.2779, не значущо.
- New_account: зниження на 7.51%, p-value = 0.0088 (значущо).

*Азія*

Жодна з метрик не показала статистично значущих змін:
- Add_payment_info: зростання на 3.47%, p-value = 0.5702, не значущо.
- Add_shipping_info: зростання на 6.26%, p-value = 0.2190, не значущо.
- Begin_checkout: зростання на 6.44%, p-value = 0.1525, не значущо.
- New_account: зростання на 3.40%, p-value = 0.4573, не значущо.

*Африка*

Результати мають низьку достовірність через нерівномірний розподіл користувачів та малий обсяг даних, однак є значуще зниження для Begin_checkout:
- Add_payment_info: зростання на 31.65%, p-value = 0.3582, не значущо.
- Add_shipping_info: зниження на 28.83%, p-value = 0.1481, не значущо.
- Begin_checkout: зниження на 36.31%, p-value = 0.0309 (значущо).
- New_account: зростання на 10.01%, p-value = 0.6694, не значущо.

*Океанія*

Результати мають низьку достовірність через малий обсяг даних, проте виявлено значуще покращення для Begin_checkout:
- Add_payment_info: зростання на 13.65%, p-value = 0.6798, не значущо.
- Add_shipping_info: зростання на 44.72%, p-value = 0.1267, не значущо.
- Begin_checkout: зростання на 59.96%, p-value = 0.0199 (значущо).
- New_account: зростання на 22.28%, p-value = 0.3461, не значущо.

Загальні висновки:
- Європа показала найбільший позитивний вплив змін, особливо для Add_payment_info, Add_shipping_info та Begin_checkout.
- Америка не продемонструвала значущих покращень, проте New_account знизився.
- Азія не показала статистично значущих змін.
- Африка та Океанія мають малий обсяг даних, тому результати менш достовірні. В Африці значуще зниження Begin_checkout, а в Океанії значуще покращення Begin_checkout.

Рекомендується впровадити зміни для Європи, оскільки ефект значний і статистично підтверджений та переглянути вплив змін в Америці, особливо через зниження New_account. Аналіз для Африки та Океанії потребує додаткових досліджень через малий розмір вибірки. Загалом, тест показав найбільшу ефективність для користувачів у Європі та частково в Океанії, тоді як результати в Африці та Америці можуть потребувати коригувань.




## Створення фінального файлу з результатами для подальшої візуалізації.

In [ ]:
# Функція для розрахунку статистики та додавання результатів у загальний список
def calculate_and_append_results(df, test_name, category_name, results):
    metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']

    for metric in metrics:
        # Витягуємо сесії та метрики для обох груп
        session_1 = df.loc[df["event_name"] == "session", "test_group1"].iloc[0]
        session_2 = df.loc[df["event_name"] == "session", "test_group2"].iloc[0]
        event_1 = df.loc[df["event_name"] == metric, "test_group1"].iloc[0]
        event_2 = df.loc[df["event_name"] == metric, "test_group2"].iloc[0]

        # Обчислення конверсій
        conversion_rate_1 = event_1 / session_1
        conversion_rate_2 = event_2 / session_2
        metric_change = conversion_rate_2 / conversion_rate_1 - 1

        # Виконуємо Z-тест
        z_stat, p_value = sm.stats.proportions_ztest([event_1, event_2], [session_1, session_2])
        significant = p_value < 0.05  # Чи значуща відмінність

        # Додаємо результат у список
        results.append({
            "Test Name": test_name,
            "Category": category_name,
            "Metric": metric,
            "Numerator_1": "Session",  # Вказуємо, що це сесії для групи 1
            "Denominator_1": metric,
            "Numerator_1_value": f"{session_1}",
            "Denominator_1_value": f"{event_1}",
            "Numerator_2": "Session",  # Вказуємо, що це сесії для групи 2
            "Denominator_2": metric,
            "Numerator_2_value": f"{session_2}",
            "Denominator_2_value": f"{event_2}",
            "Group 1 Conversion": f"{conversion_rate_1:.2%}",
            "Group 2 Conversion": f"{conversion_rate_2:.2%}",
            "Metric Change": f"{metric_change:.2%}",
            "Z-Statistic": round(z_stat, 4),
            "P-Value": round(p_value, 4),
            "Significant": significant
        })

# Список для зберігання всіх результатів
all_results = []

# Тести для тотального аналізу
tests = {
    "Test1": grouped_df1,
    "Test2": grouped_df2,
    "Test3": grouped_df3,
    "Test4": grouped_df4
}

# Зберігаємо результати для кожного тесту в тоталі
for test_name, df in tests.items():
    calculate_and_append_results(df, test_name, "Total", all_results)

# Аналіз по каналах
channel_datasets = {
    "Direct": grouped_df1_direct,
    "Organic Search": grouped_df1_os,
    "Social Search": grouped_df1_ss,
    "Paid Search": grouped_df1_ps
}
for channel, df in channel_datasets.items():
    calculate_and_append_results(df, "Test1", channel, all_results)

# Аналіз по пристроях
device_datasets = {
    "Desktop": grouped_df1_desktop,
    "Mobile": grouped_df1_mobile,
    "Tablet": grouped_df1_tablet
}
for device, df in device_datasets.items():
    calculate_and_append_results(df, "Test1", device, all_results)

# Аналіз по континентах
continent_datasets = {
    "Europe": grouped_df1_eu,
    "America": grouped_df1_am,
    "Asia": grouped_df1_as,
    "Africa": grouped_df1_af,
    "Oceania": grouped_df1_oc
}
for continent, df in continent_datasets.items():
    calculate_and_append_results(df, "Test1", continent, all_results)

# Створюємо DataFrame і записуємо у CSV один файл
results_df = pd.DataFrame(all_results)
results_df.to_csv('all_ab_test_results.csv', index=False)

In [ ]:
from google.colab import files
# Завантаження файлу
files.download('all_ab_test_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Посилання на результати аналізу.

### Посилання на дашборд в Tableau.

https://public.tableau.com/views/ABTestingTool/Dashboard1?:language=en-GB&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

### Посилання на csv файл з результатами розрахунків.
https://docs.google.com/spreadsheets/d/1HiiG41VCCVNTdXUslgSXVy4NNMPIV1AXS_pHPF8Wnic/edit?usp=sharing